In [ ]:
#!python -m spacy download en_vectors_web_lg

In [1]:
# https://www.kaggle.com/c/whiskey-reviews-ds20/overview

import pandas as pd
import re
from collections import Counter
import spacy

nlp = spacy.load('en_vectors_web_lg')

OSError: ignored

In [31]:
# Loading in the data from the Kaggle webpage

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

First, let's dive into the data:

In [32]:
# Splitting into our three ratings in order to compare text data

mask1 = train['ratingCategory'] == 0 # excellent
mask2 = train['ratingCategory'] == 1 # good
mask3 = train['ratingCategory'] == 2 # poor

In [33]:
excellent_df = train[mask1]
good_df = train[mask2]
poor_df = train[mask3]

In [34]:
def tokenize(text):
  tokens = []
  for item in (text.replace('\n', '').lower().replace('-', ' ').split()):
    tokens.append(re.sub('[^a-z ]', '', item))
  
  return tokens

In [35]:
# Cleaning and tokenizing text data for each rating dataframe
# using the above function

excellent_df['tokens'] = excellent_df['description'].apply(tokenize)
good_df['tokens'] = good_df['description'].apply(tokenize)
poor_df['tokens'] = poor_df['description'].apply(tokenize)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [36]:
excellent_df = excellent_df.reset_index().drop(columns='index')
good_df = good_df.reset_index().drop(columns='index')
poor_df = poor_df.reset_index().drop(columns='index')

In [37]:
# Feature engineering a column that is simply a count of the tokens 
# in each row

def token_count(token_list):
  return len(token_list)


excellent_df['num_of_tokens'] = excellent_df['tokens'].apply(token_count)
good_df['num_of_tokens'] = good_df['tokens'].apply(token_count)
poor_df['num_of_tokens'] = poor_df['tokens'].apply(token_count)

In [38]:
excellent_df.head(1)

,id,description,ratingCategory,tokens,num_of_tokens
0,3861,\nAn uncommon exclusive bottling of a 6 year o...,0,"[an, uncommon, exclusive, bottling, of, a, , y...",82


In [39]:
excellent_ave_token_count = sum(excellent_df['num_of_tokens']) / len(excellent_df['num_of_tokens'])
good_ave_token_count = sum(good_df['num_of_tokens']) / len(good_df['num_of_tokens'])
poor_ave_token_count = sum(poor_df['num_of_tokens']) / len(poor_df['num_of_tokens'])

In [40]:
# So, no clear difference in number of tokens in relation to ratings, but 
# will check again once stopwords are removed..

print(excellent_ave_token_count)
print(good_ave_token_count)
print(poor_ave_token_count)

71.35144609991235
71.82367233599444
72.03076923076924


In [41]:
def count(df):
  counts = {}
  for i in range (len(df)):
    for key,val in dict(Counter(df['tokens'][i])).items():
      if key in counts:
        counts[key] += val
      else:
        counts[key] = val
  
  return counts

In [42]:
# Counting the amount of times each INDIVIDUAL token shows up in each dataframe
# using the above function.  Can use this to find more accurate stopwords for 
# this dataset

excellent_count = count(excellent_df)
good_count = count(good_df)
poor_count = count(poor_df)

In [43]:
print(len(excellent_count))
print(len(good_count))
print(len(poor_count))

7336
11089
1363


In [44]:
excellent_count = sorted(excellent_count.items(), key=lambda x: x[1], reverse=True)
good_count = sorted(good_count.items(), key=lambda x: x[1], reverse=True)
poor_count = sorted(poor_count.items(), key=lambda x: x[1], reverse=True)

In [45]:
excellent_count[:10]

[('and', 3936),
 ('the', 3900),
 ('a', 2895),
 ('of', 2271),
 ('with', 1611),
 ('is', 1284),
 ('in', 1227),
 ('', 1178),
 ('to', 934),
 ('this', 910)]

In [46]:
def standardize(dictionary):
  total = 0
  new_dict = {}
  for key,val in dictionary:
    total += val
  
  for key,val in dictionary:
    new_dict[key] = val/total
  
  return new_dict

In [47]:
excellent_count = sorted(standardize(excellent_count).items(), key=lambda x: x[1], reverse=True)
good_count = sorted(standardize(good_count).items(), key=lambda x: x[1], reverse=True)
poor_count = sorted(standardize(poor_count).items(), key=lambda x: x[1], reverse=True)

In [54]:
# Example (top 10 words with rating of excellent, standardized):

excellent_count[20:30]

[('notes', 0.005576573478111335),
 ('for', 0.005539723873630423),
 ('vanilla', 0.005011546209404019),
 ('that', 0.004839581388493097),
 ('more', 0.0046430501645949),
 ('from', 0.004606200560113988),
 ('fruit', 0.0043482533287476045),
 ('an', 0.003942907679457574),
 ('as', 0.0037955092615339263),
 ('whiskey', 0.003697243649584828)]

In [52]:
good_count[10:20]

[('to', 0.009723376698691307),
 ('finish', 0.009298099785428466),
 ('oak', 0.0077564709748506695),
 ('palate', 0.007026734453229205),
 ('nose', 0.006379153698942607),
 ('vanilla', 0.006364655622354101),
 ('notes', 0.006161682550115018),
 ('it', 0.005876553710541068),
 ('its', 0.0058427248651678875),
 ('sweet', 0.00579923063540237)]

In [55]:
poor_count[20:30]

[('palate', 0.006407518154634771),
 ('more', 0.005766766339171294),
 ('that', 0.005553182400683468),
 ('as', 0.005553182400683468),
 ('whiskey', 0.005339598462195643),
 ('hot', 0.005339598462195643),
 ('like', 0.004912430585219991),
 ('old', 0.004698846646732166),
 ('notes', 0.004698846646732166),
 ('an', 0.004698846646732166)]

In [21]:
def create_stopwords(dictionary):
  stopwords = []
  for key,val in dictionary:
    if val > 0.001:
      stopwords.append(key)
    else:
      continue
  
  return stopwords

In [22]:
stop_words = create_stopwords(excellent_count) + create_stopwords(good_count) + create_stopwords(poor_count)

In [23]:
stop_words = set(stop_words)

In [24]:
def find_anomalies(dictionary, percent_of):
  common = []
  for key,val in dictionary[:percent_of]:
      common.append(key)
  
  return common

In [25]:
top_excellent = find_anomalies(excellent_count, int(len(excellent_count)*1))
top_good = find_anomalies(good_count, int(len(good_count)*1))
top_poor = find_anomalies(poor_count, int(len(poor_count)*1))

In [26]:
# Lists of unique words in each rating category:

In [27]:
unique_excellent = list(set(top_excellent) - set(top_good) - set(top_poor))
unique_good = list(set(top_good) - set(top_excellent) - set(top_poor))
unique_poor = list(set(top_poor) - set(top_excellent) - set(top_good))

In [28]:
print(unique_excellent)
print(unique_good)
print(unique_poor)

['waited', 'bounds', 'floridas', 'precipitously', 'oilbut', 'bolt', 'grapey', 'fusty', 'pigskin', '190500', 'custodians', 'chassagne', 'theft', 'selecting', 'dump', 'buoyed', 'screech', 'montana', 'woozy', 'cave', 'options', 'calmed', 'dissolving', 'riposte', 'gilead', 'gelephu', 'refreshes', 'sloe', 'expand', 'pewaukee', 'refuse', '1870', 'vinhos', 'standpoint', 'constricted', 'honeyedtoffee', '1221', 'smoothest', 'voce', 'proportioned', 'suduiraut', 'curaao', 'darkroom', 'forays', 'kgb', 'presumably', 'throughbaked', 'wheelbarrow', 'harry', 'hurdle', 'bagels', 'tugs', 'maids', 'smoothly', 'sprigs', 'trophy', 'gingko', 'proximity', 'baguette', 'freesia', 'nicknamed', 'newfies', 'flowersstems', 'latent', 'reimagined', 'schramm', 'mangoemerging', 'soaking', 'c40375', 'voice', 'sushi', 'chf89', 'piled', 'movie', 'willies', 'cringe', 'father', 'powdering', 'nostalgic', 'voices', 'omoray', 'alley', 'enterprising', '2018s', 'erodes', 'kate', 'mineraly', 'merits', 'pretend', 'montrose', 'fru

In [29]:
unique_words = set(unique_excellent + unique_good + unique_poor)

In [30]:
len(unique_words)

8301

In [140]:
def vectorize(token_list):
  vects = []
  doc = nlp(str(' '.join(token_list)))

  for token in doc:
    if token.vector_norm > 0.0:
      vects.append(token.vector_norm)
    else:
      continue
  return vects

In [141]:
excellent_df['vectors'] = excellent_df['tokens'].apply(vectorize)
good_df['vectors'] = good_df['tokens'].apply(vectorize)
poor_df['vectors'] = poor_df['tokens'].apply(vectorize)

In [142]:
def ave_vects(vects_list):
  return sum(vects_list) / len(vects_list)

In [143]:
excellent_df['vects_ave'] = excellent_df['vectors'].apply(ave_vects)
good_df['vects_ave'] = good_df['vectors'].apply(ave_vects)
poor_df['vects_ave'] = poor_df['vectors'].apply(ave_vects)

In [144]:
excellent_df.head(1)

,id,description,ratingCategory,tokens,num_of_tokens,vectors,vects_ave
0,3861,\nAn uncommon exclusive bottling of a 6 year o...,0,"[an, uncommon, exclusive, bottling, of, a, 6, ...",82,"[5.5624833, 4.9866605, 5.9754963, 7.040095, 4....",5.951116


In [145]:
print(sum(excellent_df['vects_ave']) / len(excellent_df['vects_ave']))
print(sum(good_df['vects_ave']) / len(good_df['vects_ave']))
print(sum(poor_df['vects_ave']) / len(poor_df['vects_ave']))

5.982035087103118
6.080487899267268
5.879506043032742


Alright, let's apply all of this to the training data!

In [146]:
# Adding anomalies to tokenize function

def tokenize(text):
  tokens = []
  for item in (text.replace('\n', '').lower().replace('-', ' ').split()):
    tokens.append(re.sub('[^a-z 0-9]', '', item))
  
  unique_words
  cleaned_tokens = []

  for word in tokens:
    if word not in unique_words:
      cleaned_tokens.append(word)
    else:
      continue

  return cleaned_tokens

In [147]:
def wrangle(df):
  df['tokens'] = df['description'].apply(tokenize)
  df['num_of_tokens'] = df['tokens'].apply(token_count)
  df['vectors'] = df['tokens'].apply(vectorize)
  df['vects_ave'] = df['vectors'].apply(ave_vects)

  return df

In [148]:
train = wrangle(train)

In [149]:
train.head(1)

,id,description,ratingCategory,tokens,num_of_tokens,vectors,vects_ave
0,1321,"\nSometimes, when whisky is batched, a few lef...",1,"[sometimes, when, whisky, is, a, few, barrels,...",74,"[5.4366913, 4.824553, 7.2341533, 4.890306, 5.3...",6.010289


In [150]:
# Split into target and feature matrix

X = train.drop(columns=['id','description','ratingCategory','tokens','num_of_tokens','vectors'])
y = train['ratingCategory']

print(X.shape)
print(y.shape)

assert len(X) == len(y)

(4087, 1)
(4087,)


In [151]:
X.head()

,vects_ave
0,6.010289
1,5.951116
2,5.978484
3,6.258276
4,6.261551


In [152]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=42)

clf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [153]:
test1 = wrangle(test).drop(columns=['id','description','tokens','num_of_tokens','vectors'])

In [154]:
test1.head()

,vects_ave
0,6.351125
1,5.972300
2,6.172125
3,6.406283
4,6.042514


In [155]:
pred = clf.predict(test1)

In [156]:
pred

array([1, 1, 1, ..., 1, 1, 1])

In [157]:
test_id = pd.read_csv('test.csv')

In [158]:
submission = pd.DataFrame({'id': test_id['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [159]:
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,0


In [160]:
submission['ratingCategory'].value_counts()

1    715
0    288
2     19
Name: ratingCategory, dtype: int64

In [161]:
subNumber = 11
submission.to_csv(f'submission{subNumber}.csv', index=False)